<a href="https://colab.research.google.com/github/junwin/colab/blob/main/MLSDataClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean and convert MLS data exported to excel workbooks

This utility reads one or more Excel workbooks of data exported from the MLS and outputs a CSV file that can be used to train a Tensorflow model.

In [151]:
#@title Dependancies
import pandas as pd

import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import datetime
import os

In [152]:
#url="/content/MLSReportDetachedGlenview.csv"
#rawData=pd.read_csv(url).sample(frac=1)
#rawData.describe()

In [153]:
cleanData = pd.DataFrame()

#MLS#	Stat	ClosedDate	YearClosed	SoldPr	Type	YearBuilt	Street#	StrName	Locale	Zip	Area	Rooms	FullBaths	HalfBaths	BsmtBth	Beds	BsmtBeds	GarageType	GarageSpaces	ParkingSpaces
requiredColumns = ['MLS #','Stat','ClosedDate','YearClosed','SoldPr','Type','YearBuilt','Street#','StrName','Locale','Zip','Area','Rooms','FullBaths','HalfBaths','BsmtBth','Beds','BsmtBeds','GarageType','GarageSpaces','ParkingSpaces']
houseInfo =  pd.DataFrame( columns=requiredColumns)

#Functions and main data processing for each file

In [154]:
def cleanType(houseDataRow, filePath):
  if 'detached' in filePath.lower():
    houseDataRow['Type'] = 'SFH'
  elif 'condo' in houseDataRow['Type'].lower():
    houseDataRow['Type'] = 'Condo'
  elif 'townhouse' in houseDataRow['Type'].lower():
    houseDataRow['Type'] = 'Townhouse'
  elif 'duplex' in houseDataRow['Type'].lower():
    houseDataRow['Type'] = 'Duplex'
  else:
    return False

  return True

def cleanDates(houseDataRow):
  if not pd.isnull( houseDataRow['ClosedDate']):
    #date_time_obj = datetime.datetime.strptime(houseDataRow['ClosedDate'], '%m/%d/%Y')
    houseDataRow['YearClosed'] = houseDataRow['ClosedDate'].year

def cleanGarageData(houseDataRow):
  if pd.isnull( houseDataRow['GarageSpaces']) or pd.isna( houseDataRow['GarageSpaces']):
    houseDataRow['GarageSpaces'] = 0
  if pd.isnull( houseDataRow['ParkingSpaces']) or pd.isna( houseDataRow['ParkingSpaces']):
    houseDataRow['ParkingSpaces'] = 0
  if pd.isnull( houseDataRow['GarageType']) or pd.isna( houseDataRow['GarageType']):
    houseDataRow['GarageType'] = 'None'


def processMlsCsvFile(filePath, houseInfo):
  #url="/content/MLSReportDetachedGlenview.csv"
  #rawData=pd.read_csv(filePath).sample(frac=1)
  rawData=pd.read_excel(filePath,header=0)
  #rawData.describe()
  for i, row in rawData.iterrows():
    numBeds=0
    if 'Beds' in rawData.columns:
      numBeds=row['Beds']
    elif 'All Beds' in rawData.columns:
      numBeds = row['All Beds']
    else:
      continue

    if not str.isnumeric(str(numBeds)):
      continue

    houseData = {'MLS #': row['MLS #'],
    'Stat': row['Stat'],
    'ClosedDate': row['Closed Date'],
    'YearClosed': row['Closed Date'],
    'SoldPr': row['Sold Pr'],
    'Type': row['Type'],
    'YearBuilt': row['Yr Blt'],
    'Street#': row['Street #'],
    'StrName': row['Str Name'],
    'Locale': row['Area'],
    'Zip': row['Zip'],
    'Area': row['ASF'],
    'Rooms': row['# Rms'],
    'FullBaths': row['# Full Baths'],
    'HalfBaths': row['# Half Baths'],
    'BsmtBth': row['Bsmt Bth'],
    'Beds': numBeds,
    'BsmtBeds': row['Bsmt. Beds'],
    'GarageType': row['Garage Type'],
    'GarageSpaces': row['# Garage Spaces'],
    'ParkingSpaces': row['# Parking Spaces'] }

    if pd.isnull( houseData['Area']) or pd.isna( houseData['Area']) or houseData['Area'] == 0 :
      continue
    if pd.isnull( houseData['BsmtBth']) or pd.isna( houseData['BsmtBth']):
      houseData['BsmtBth']='No'

    
    if not cleanType(houseData, filePath):
      continue
    
    cleanGarageData(houseData)
    
    cleanDates(houseData)
   
    houseInfo = houseInfo.append(pd.Series(houseData), ignore_index=True)


  return houseInfo




#Open files and process data

In [155]:
houseInfo =  pd.DataFrame( columns=requiredColumns)
#/content/MLSReportDetachedGlenview.csv
path = '/content/'
fileList = os.listdir(path)
for i in fileList:
    if i.endswith(".xlsx"):
      filePath = path + i
      print(filePath)
      #addedRows = processMlsCsvFile(filePath, houseInfo)
      #houseInfo = houseInfo.append(addedRows)
      houseInfo = processMlsCsvFile(filePath, houseInfo)


houseInfo.to_csv('/content/clean.csv', index=False)


/content/MLS Report Detached Morton Grove_2019 and 2020.xlsx
/content/MLS Report Attached Wilmette_2019 and 2020.xlsx
/content/MLS Report Attached Morton Grove_2019 and 2020.xlsx
/content/MLS Report Attached Glenview_2019 and 2020.xlsx
/content/MLS Report Detached Glenview_2019 and 2020.xlsx
/content/MLS Report Detached Wilmette_2019 and 2020.xlsx
